In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

c:\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [5]:
y2015['loan_status'].unique()

array(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)',
       'In Grace Period', 'Default', 'Late (16-30 days)'], dtype=object)

In [6]:
#0 = Current 
#1 = Fully Paid
#2 = Charged Off
#3 = Late (31-120 days)
#4 = In Grace Period
#5 = Default
#6 = Late (16-30 days)

mapping = {'Current': 0, 'Fully Paid': 1, 'Charged Off': 2, 
           'Late (31-120 days)': 3, 'In Grace Period': 4, 'Default': 5,
          'Late (16-30 days)': 6}

y2015['loan_status'].replace(mapping, inplace=True)



In [7]:
X = pd.get_dummies(y2015)
correlated = abs(X.corr()['loan_status']).sort_values(ascending=False)

In [8]:
most_correlated = []

for i in correlated.index:
    if correlated[i] >= .1 and correlated[i] < 1:
        most_correlated.append(i)

most_correlated

['last_pymnt_d_Jan-2017',
 'next_pymnt_d_Feb-2017',
 'last_credit_pull_d_Jan-2017',
 'out_prncp_inv',
 'out_prncp',
 'last_credit_pull_d_Oct-2016',
 'last_pymnt_amnt',
 'collection_recovery_fee',
 'recoveries',
 'last_pymnt_d_Sep-2016',
 'last_pymnt_d_Nov-2016',
 'last_pymnt_d_Oct-2016',
 'int_rate',
 'last_pymnt_d_Aug-2016',
 'last_pymnt_d_Jul-2016',
 'last_pymnt_d_Jun-2016',
 'last_pymnt_d_May-2016',
 'last_pymnt_d_Mar-2016',
 'total_rec_late_fee',
 'last_pymnt_d_Apr-2016',
 'acc_open_past_24mths',
 'num_tl_op_past_12m',
 'last_credit_pull_d_Dec-2016',
 'last_pymnt_d_Feb-2016',
 'next_pymnt_d_Jan-2017',
 'last_pymnt_d_Jan-2016',
 'last_pymnt_d_Dec-2015',
 'grade_A']

In [9]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = X[most_correlated]
Y = y2015['loan_status']
X = X.dropna(axis=1)

scores = cross_val_score(rfc, X, Y, cv=10)

c:\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default valu

In [10]:
scores

array([0.97243001, 0.97214505, 0.97364109, 0.97406853, 0.97195441,
       0.97442413, 0.97252369, 0.97344986, 0.97014748, 0.97271173])